In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import geopandas as gpd

sys.path.append("../")
from src.utils import sample_points, sample_categories

In [2]:
# specify paths to wildfire ignitions and forest type
path_to_ignition_points = r"C:\Users\David\Documents\ZGIS\Nextcloud_MyFiles\Projects\IGNITE\data\raw\BOKU_Fire_Ignitions\boku_wildfire_ignitions.shp"

path_to_forest_type_layer = r"C:\Users\David\Documents\ZGIS\Nextcloud_MyFiles\Projects\IGNITE\data\raw\BWF_forest_type\forest_type_merged.tif"

In [3]:
# read in wildfire ignition data
ignition_points = gpd.read_file(path_to_ignition_points)
ignition_points.head(2)

,WaldbrandI,Datum,UTC,Bundesland,WGS84_long,WGS84_lat_,Pufferradi,Ursache1,Near_build,N_people,Dist_roads,Roads,Rad_dist,aspect_aus,dhm_at_lam,slope_aust,Veg1,Veg2_1,Veg3_firEU,geometry
0,2021052701,5/27/2021,15:00,NOE,16.133164,47.877681,300,anthropogen,50.368667,7,54.166028,sonst,237.233615,326.15982,367.75150,11.21131,Pine_high_,Pine_high_,Urban,POINT (4779428.082 2770595.296)
1,2021052102,5/21/2021,7:50,NOE,16.181795,47.778698,50,anthropogen,207.367038,0,10.398109,for_road,8.045477,50.58361,303.09656,0.44824,Pine_high_,Pine_high_,Crop,POINT (4783946.969 2759929.697)


In [4]:
# print information of wildfire ignition dataset
ignition_points.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1402 entries, 0 to 1401
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   WaldbrandI  1402 non-null   int64   
 1   Datum       1402 non-null   object  
 2   UTC         1402 non-null   object  
 3   Bundesland  1402 non-null   object  
 4   WGS84_long  1402 non-null   float64 
 5   WGS84_lat_  1402 non-null   float64 
 6   Pufferradi  1402 non-null   int64   
 7   Ursache1    1402 non-null   object  
 8   Near_build  1402 non-null   float64 
 9   N_people    1402 non-null   int64   
 10  Dist_roads  1402 non-null   float64 
 11  Roads       1402 non-null   object  
 12  Rad_dist    1402 non-null   float64 
 13  aspect_aus  1402 non-null   float64 
 14  dhm_at_lam  1402 non-null   float64 
 15  slope_aust  1402 non-null   float64 
 16  Veg1        1402 non-null   object  
 17  Veg2_1      1402 non-null   object  
 18  Veg3_firEU  1402 non-null   object  
 19

In [5]:
# select human caused fires for training dataset creation
ignition_points_human = ignition_points[ignition_points["Ursache1"] == "anthropogen"]

In [6]:
# sample random points from Forest Type layer (non-ignition points) - as many as ignition points
nr_points = len(ignition_points_human)
non_ignition_points = sample_points(path_to_forest_type_layer, nr_points)

In [ ]:
# prepare fire ignition points for merge
ignition_points["year"] = ignition_points.Datum.apply(lambda x: x.split("/")[-1])
ignition_points["month"] = ignition_points.Datum.apply(lambda x: x.split("/")[0])

In [ ]:
ignition_points["fire"] = 1

# the reference system the data set should be transformed to
target_crs = "EPSG:31287"
ignition_points = ignition_points.to_crs(target_crs)

In [ ]:
# prepare non ignition points for merge
non_ignition_points["fire"] = 0
non_ignition_points["Pufferradi"] = None

In [ ]:
# sample years according to the occurence distribution in the fire ignition dataset
cat_probs = dict(ignition_points["month"].value_counts())
categories = list(cat_probs.keys())
probabilities = [prob / len(non_ignition_points) for prob in list(cat_probs.values())]

In [ ]:
categories_nr = [int(i) for i in categories]

In [ ]:
month_samples = sample_categories(categories_nr, probabilities, len(non_ignition_points))
non_ignition_points["month"] = month_samples
non_ignition_points = non_ignition_points.to_crs(target_crs)

In [ ]:
# sample year and day randomly
